# Importing Libraries

In [0]:
# Import the required libraries.
import numpy as np 
import pandas as pd 
import nltk
import os
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply,LSTMCell,RNN,BatchNormalization
from keras.layers import RepeatVector, Dense, Activation, Lambda, Reshape,TimeDistributed,Concatenate
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
from keras import metrics

# Encoder-Decoder Models for Both the Translations

# English to Hindi

## Data Preparation

In [0]:
# Load the dataset.
dataset=pd.read_csv('train_english.en',names= ['english'],sep='\t',header=None)
dataset1=pd.read_csv('train_hindi.hi',names= ['hindi'],sep='\t',header=None)

test_dataset = pd.read_csv('test_english.en',names= ['english'],sep='\t',header=None)
test_dataset1=pd.read_csv('test_hindi.hi',names= ['hindi'],sep='\t',header=None)

In [0]:
dataset.head()

,english
0,give your application an accessibility workout
1,accerciser accessibility explorer
2,the default plugin layout for the bottom panel
3,the default plugin layout for the top panel
4,a list of plugins that are disabled by default


In [0]:
dataset1.head()

,hindi
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक
2,निचले पटल के लिए डिफोल्ट प्लग इन खाका
3,ऊपरी पटल के लिए डिफोल्ट प्लग इन खाका
4,उन प्लग इनों की सूची जिन्हें डिफोल्ट रूप से नि...


In [0]:
dataset['hindi'] = dataset1['hindi']
test_dataset['hindi'] = test_dataset1['hindi']

In [0]:
# Add sentence beginning and end markers.
dataset['english']=dataset['english'].apply(lambda x:x+' $')
dataset['hindi']=dataset['hindi'].apply(lambda x:'^ '+x+' $')
dataset.head()

test_dataset['english']=test_dataset['english'].apply(lambda x:x+' $')
test_dataset['hindi']=test_dataset['hindi'].apply(lambda x:'^ '+x+' $')


In [0]:
test_dataset.head()

,english,hindi
0,a black box in your car $,^ आपकी कार में ब्लैक बॉक्स $
1,as americas road planners struggle to find the...,^ जबकि अमेरिका के सड़क योजनाकार ध्वस्त होते हु...
2,the devices which track every mile a motorist ...,^ यह डिवाइस जो मोटर चालक द्वारा वाहन चलाए गए प...
3,the usually dull arena of highway planning has...,^ आम तौर पर हाईवे नियोजन जैसा उबाऊ काम भी अचान...
4,libertarians have joined environmental groups ...,^ आपने द्वारा ड्राइव किए गए मील तथा संभवतः ड्र...


## Data Preprocessing

In [0]:
X=dataset.english.tolist() + test_dataset.english.tolist()
Y=dataset.hindi.tolist() + test_dataset.hindi.tolist()

In [0]:
print(Y)

['^ अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें $', '^ एक्सेर्साइसर पहुंचनीयता अन्वेषक $', '^ निचले पटल के लिए डिफोल्ट प्लग इन खाका $', '^ ऊपरी पटल के लिए डिफोल्ट प्लग इन खाका $', '^ उन प्लग इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है $', '^ अवधि को हाइलाइट रकें $', '^ पहुंचनीय आसंधि नोड को चुनते समय हाइलाइट बक्से की अवधि $', '^ सीमांत बोर्डर के रंग को हाइलाइट करें $', '^ हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। $', '^ भराई के रंग को हाइलाइट करें $', '^ हाइलाइट किया गया भराई का रंग और पारदर्शिता। $', '^ एपीआई विचरक $', '^ इस समय जिसे प्राप्त किया गया हो उसकी विभिन्न विधियों मेथड में विचरण करें $', '^ निजी गुणों को छिपाएं $', '^ विधि $', '^ गुणधर्म $', '^ मान $', '^ आईपाइथन कन्सोल $', '^ इस समय चुने गए एक्सेसेबेल से काम लेने के लिए अंतर्क्रियात्मक कन्सोल $', '^ घटना मानिटर $', '^ घटनाओं को मानिटर करें $', '^ चुनाव को हटाएं $', '^ सभी $', '^ चुने गए अनुप्रयोग $', '^ चुने गए एक्सेसेबेल $', '^ स्रोत $', '^ चुने गए प्रकारों और स्रोतों से घटनाएं जैसे जैसे घटित होती हैं 

In [0]:
# Compute word counts.
X_words_counts={}
for row in X:
    for words in row.split(' '):
        #print(words)
        X_words_counts[words]=X_words_counts.get(words,0)+1

In [0]:
# Select the most common words.
most_common_X_words = sorted(X_words_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_X_words)

[('$', 2267), ('the', 619), ('of', 409)]


In [0]:
Y_words_counts={}
for row in Y:
    for words in row.split(' '):
        #print(words)
        Y_words_counts[words]=Y_words_counts.get(words,0)+1

In [0]:
most_common_Y_words = sorted(Y_words_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_Y_words)

[('^', 2267), ('$', 2267), ('को', 366)]


In [0]:
cnt=0
X_WORDS_TO_INDEX={}
for w in X_words_counts:
    X_WORDS_TO_INDEX[w] =cnt
    cnt+=1
X_WORDS_TO_INDEX['#']=len(X_WORDS_TO_INDEX)
ALL_X_WORDS = X_WORDS_TO_INDEX.keys()
print(X_WORDS_TO_INDEX)

{'give': 0, 'your': 1, 'application': 2, 'an': 3, 'accessibility': 4, 'workout': 5, '$': 6, 'accerciser': 7, 'explorer': 8, 'the': 9, 'default': 10, 'plugin': 11, 'layout': 12, 'for': 13, 'bottom': 14, 'panel': 15, 'top': 16, 'a': 17, 'list': 18, 'of': 19, 'plugins': 20, 'that': 21, 'are': 22, 'disabled': 23, 'by': 24, 'highlight': 25, 'duration': 26, 'box': 27, 'when': 28, 'selecting': 29, 'accessible': 30, 'nodes': 31, 'border': 32, 'color': 33, 'and': 34, 'opacity': 35, 'fill': 36, 'api': 37, 'browser': 38, 'browse': 39, 'various': 40, 'methods': 41, 'current': 42, 'hide': 43, 'private': 44, 'attributes': 45, 'method': 46, 'property': 47, 'value': 48, 'ipython': 49, 'console': 50, 'interactive': 51, 'manipulating': 52, 'currently': 53, 'selected': 54, 'event': 55, 'monitor': 56, 'events': 57, 'clear': 58, 'selection': 59, 'everything': 60, 'source': 61, 'shows': 62, 'as': 63, 'they': 64, 'occur': 65, 'from': 66, 'types': 67, 'sources': 68, 'last': 69, 'entry': 70, 'start': 71, 'stop

In [0]:
cnt=0
Y_WORDS_TO_INDEX={}
for w in Y_words_counts:
    Y_WORDS_TO_INDEX[w] =cnt
    cnt+=1
Y_WORDS_TO_INDEX['#']=len(Y_WORDS_TO_INDEX)
ALL_Y_WORDS = Y_WORDS_TO_INDEX.keys()
print(Y_WORDS_TO_INDEX)

{'^': 0, 'अपने': 1, 'अनुप्रयोग': 2, 'को': 3, 'पहुंचनीयता': 4, 'व्यायाम': 5, 'का': 6, 'लाभ': 7, 'दें': 8, '$': 9, 'एक्सेर्साइसर': 10, 'अन्वेषक': 11, 'निचले': 12, 'पटल': 13, 'के': 14, 'लिए': 15, 'डिफोल्ट': 16, 'प्लग': 17, 'इन': 18, 'खाका': 19, 'ऊपरी': 20, 'उन': 21, 'इनों': 22, 'की': 23, 'सूची': 24, 'जिन्हें': 25, 'रूप': 26, 'से': 27, 'निष्क्रिय': 28, 'किया': 29, 'गया': 30, 'है': 31, 'अवधि': 32, 'हाइलाइट': 33, 'रकें': 34, 'पहुंचनीय': 35, 'आसंधि': 36, 'नोड': 37, 'चुनते': 38, 'समय': 39, 'बक्से': 40, 'सीमांत': 41, 'बोर्डर': 42, 'रंग': 43, 'करें': 44, 'किए': 45, 'गए': 46, 'और': 47, 'अपारदर्शिता।': 48, 'भराई': 49, 'पारदर्शिता।': 50, 'एपीआई': 51, 'विचरक': 52, 'इस': 53, 'जिसे': 54, 'प्राप्त': 55, 'हो': 56, 'उसकी': 57, 'विभिन्न': 58, 'विधियों': 59, 'मेथड': 60, 'में': 61, 'विचरण': 62, 'निजी': 63, 'गुणों': 64, 'छिपाएं': 65, 'विधि': 66, 'गुणधर्म': 67, 'मान': 68, 'आईपाइथन': 69, 'कन्सोल': 70, 'चुने': 71, 'एक्सेसेबेल': 72, 'काम': 73, 'लेने': 74, 'अंतर्क्रियात्मक': 75, 'घटना': 76, 'मानिटर': 77, 'घटनाओं'

In [0]:
def length_of_sentence(sentence):
    return len(sentence.split(' '))
dataset['e_length']=dataset['english'].apply(length_of_sentence)
dataset['h_length']=dataset['hindi'].apply(length_of_sentence)
Tx=dataset.e_length.max()
Ty=dataset.h_length.max()
print(Tx)
print(Ty)

40
40


In [0]:
def length_of_sentence(sentence):
    return len(sentence.split(' '))
test_dataset['e_length']=test_dataset['english'].apply(length_of_sentence)
test_dataset['h_length']=test_dataset['hindi'].apply(length_of_sentence)
Txx=test_dataset.e_length.max()
Tyy=test_dataset.h_length.max()
print(Txx)
print(Tyy)

49
61


In [0]:
Tx = max(Tx,Txx)
Ty = max(Ty,Tyy)

In [0]:
def series_to_array(series,vocab,length):
    X_train=[]
    for row in series:
        r=row.split(' ')
        R=[]
        for a in r:
            R.append(vocab[a])
        X_train.append(R)
    # length = max(map(len, X_train))
    y=np.array([xi+[None]*(length-len(xi)) for xi in X_train])
    return y

In [0]:
X_train=series_to_array(dataset.english,X_WORDS_TO_INDEX,Tx)
X_train=np.where(X_train==None, len(X_WORDS_TO_INDEX)-1, X_train)
print(X_train.shape)

(2167, 49)


In [0]:
Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(X_WORDS_TO_INDEX)), X_train)))
print(Xoh.shape)

Using TensorFlow backend.


(2167, 49, 1950)


In [0]:
Y_train=series_to_array(dataset.hindi,Y_WORDS_TO_INDEX,Ty)
Y_train=np.where(Y_train==None, len(Y_WORDS_TO_INDEX)-1, Y_train)
print(Y_train)
Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(Y_WORDS_TO_INDEX)), Y_train)))
Yo=Yoh[:,1:,:]
print(Yo.shape)
ze=np.zeros((1,len(Y_WORDS_TO_INDEX)))
ze[0][Y_WORDS_TO_INDEX['^']]=1
Yo=np.insert(arr=Yo,obj=Ty-1,values=ze,axis=1)
print(Yo.shape)

[[0 1 2 ... 2126 2126 2126]
 [0 10 4 ... 2126 2126 2126]
 [0 12 13 ... 2126 2126 2126]
 ...
 [0 1511 9 ... 2126 2126 2126]
 [0 43 9 ... 2126 2126 2126]
 [0 1511 800 ... 2126 2126 2126]]
(2167, 60, 2127)
(2167, 61, 2127)


## Encoder- Decoder Model 

In [0]:
n_a=2048

In [0]:
encoder_inputs = Input(shape=(Tx, len(X_WORDS_TO_INDEX)))
encoder = Bidirectional(LSTM(n_a, return_state=True))
encoder_outputs, state_h_f, state_c_f,state_h_b,state_c_b = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
state_h=Concatenate()([state_h_f,state_h_b])
state_c=Concatenate()([state_c_f,state_c_b])
print(state_h.shape)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, len(Y_WORDS_TO_INDEX)))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(2*n_a, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(len(Y_WORDS_TO_INDEX), activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)
print(decoder_outputs.shape)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

(None, 4096)
(None, None, 2127)


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 49, 1950)     0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 4096), (None 65519616    input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 2127)   0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 4096)         0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]      

In [26]:
opt = Adam(lr=0.001,beta_1=0.9,beta_2=0.999,decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit([Xoh,Yoh], Yo, epochs=50, batch_size=64)

Epoch 1/50
2167/2167 [==============================] - 89s 41ms/step - loss: 2.0358 - accuracy: 0.8360
Epoch 2/50
2167/2167 [==============================] - 89s 41ms/step - loss: 1.0103 - accuracy: 0.8920
Epoch 3/50
2167/2167 [==============================] - 89s 41ms/step - loss: 1.0374 - accuracy: 0.9083
Epoch 4/50
2167/2167 [==============================] - 89s 41ms/step - loss: 0.8119 - accuracy: 0.9079
Epoch 5/50
2167/2167 [==============================] - 89s 41ms/step - loss: 0.7601 - accuracy: 0.9103
Epoch 6/50
2167/2167 [==============================] - 89s 41ms/step - loss: 0.7410 - accuracy: 0.9110
Epoch 7/50
2167/2167 [==============================] - 89s 41ms/step - loss: 0.7347 - accuracy: 0.9109
Epoch 8/50
2167/2167 [==============================] - 89s 41ms/step - loss: 0.7305 - accuracy: 0.9109
Epoch 9/50
2167/2167 [==============================] - 89s 41ms/step - loss: 0.7281 - accuracy: 0.9110
Epoch 10/50
2167/2167 [==============================] - 90s 41m

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(2*n_a,))
decoder_state_input_c = Input(shape=(2*n_a,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

## Model Testing

In [0]:
INDEX_TO_WORD_Y={y:x for (x,y) in Y_WORDS_TO_INDEX.items()}


In [29]:
X_test=series_to_array(test_dataset.english,X_WORDS_TO_INDEX,Tx)
X_test=np.where(X_test==None, len(X_WORDS_TO_INDEX)-1, X_test)
print(X_test.shape)

from keras.utils import to_categorical
Xoh_test = np.array(list(map(lambda x: to_categorical(x, num_classes=len(X_WORDS_TO_INDEX)), X_test)))
print(Xoh_test.shape)

(100, 49)
(100, 49, 1950)


In [30]:
Y_test=series_to_array(test_dataset.hindi,Y_WORDS_TO_INDEX,Ty)
Y_test=np.where(Y_test==None, len(Y_WORDS_TO_INDEX)-1, Y_test)
print(Y_test)
Yoh_test = np.array(list(map(lambda x: to_categorical(x, num_classes=len(Y_WORDS_TO_INDEX)), Y_test)))
Yo_test=Yoh_test[:,1:,:]
print(Yo_test.shape)
ze=np.zeros((1,len(Y_WORDS_TO_INDEX)))
ze[0][Y_WORDS_TO_INDEX['^']]=1
Yo_test=np.insert(arr=Yo_test,obj=Ty-1,values=ze,axis=1)
print(Yo_test.shape)

[[0 1512 1513 ... 2126 2126 2126]
 [0 1516 1517 ... 2126 2126 2126]
 [0 455 1258 ... 1555 897 9]
 ...
 [0 2110 219 ... 2126 2126 2126]
 [0 2115 1067 ... 2126 2126 2126]
 [0 2058 2121 ... 2126 2126 2126]]
(100, 60, 2127)
(100, 61, 2127)


In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(Y_WORDS_TO_INDEX)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, Y_WORDS_TO_INDEX['^']] = 1

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    cnt = 0
    while not stop_condition and cnt < 100:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = INDEX_TO_WORD_Y[sampled_token_index]
        decoded_sentence += (' ' + sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '$' or sampled_char == '#' ):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(Y_WORDS_TO_INDEX)))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
        cnt+=1

    return decoded_sentence

In [37]:
import nltk
cunt=0
Avg_bleu = 0
for index, row in test_dataset.iterrows():
    source=Xoh_test[index,:,:]
    source=source.reshape(1,Tx,len(X_WORDS_TO_INDEX))
    #print(source)
    result=decode_sequence(source)

    hypothesis = result.split(' ')[1:-1]
    reference = row['hindi'].split(' ')[1:-1]
    print(hypothesis)
    print(reference)
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
    print(BLEUscore)
    Avg_bleu += BLEUscore

    # print('##########')
    print("source:", row['english'])
    print('real:',row['hindi'])
    print("output:", result)
    # if cunt==10:
    #     break
    cunt+=1
    print('-------------------------------------------------------------')

print(Avg_bleu/100)

['निर्देश']
['आपकी', 'कार', 'में', 'ब्लैक', 'बॉक्स']
0
source: a black box in your car $
real: ^ आपकी कार में ब्लैक बॉक्स $
output:  निर्देश $
-------------------------------------------------------------
['को', 'के', 'के', 'के', 'बगल', 'में', 'में', 'के']
['जबकि', 'अमेरिका', 'के', 'सड़क', 'योजनाकार', 'ध्वस्त', 'होते', 'हुए', 'हाईवे', 'सिस्टम', 'को', 'सुधारने', 'के', 'लिए', 'धन', 'की', 'कमी', 'से', 'जूझ', 'रहे', 'हैं', 'वहीं', 'बहुत', 'से', 'लोग', 'इसका', 'समाधान', 'छोटे', 'से', 'ब्लैक', 'बॉक्स', 'में', 'देख', 'रहे', 'हैं', 'जो', 'आपकी', 'कार', 'के', 'डैशबोर्ड', 'पर', 'सफ़ाई', 'से', 'फिट', 'हो', 'जाता', 'है।']
0.006788665421552941
source: as americas road planners struggle to find the cash to mend a crumbling highway system many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car $
real: ^ जबकि अमेरिका के सड़क योजनाकार ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं वहीं बहुत से लोग इसका समाधान छोटे से ब्लैक बॉक्स 

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


['निर्देश']
['यह', 'डिवाइस', 'जो', 'मोटर', 'चालक', 'द्वारा', 'वाहन', 'चलाए', 'गए', 'प्रत्येक', 'मील', 'को', 'ट्रैक', 'करती', 'है', 'तथा', 'उस', 'सूचना', 'को', 'अधिकारियों', 'को', 'संचारित', 'करती', 'है', 'आजकल', 'अमेरिका', 'की', 'प्रमुख', 'सड़कों', 'का', 'वित्त', 'पोषण', 'करने', 'के', 'लिए', 'पुराने', 'हो', 'चुके', 'सिस्टम', 'का', 'जीर्णोद्धार', 'करने', 'के', 'लिए', 'वाशिंगटन', 'और', 'राज्य', 'नियोजन', 'कार्यालय', 'के', 'लिए', 'एक', 'विवादास्पद', 'प्रयास', 'का', 'मुद्दा', 'बन', 'चुका', 'है।']
0
source: the devices which track every mile a motorist drives and transmit that information to bureaucrats are at the center of a controversial attempt in washington and state planning offices to overhaul the outdated system for funding americas major roads $
real: ^ यह डिवाइस जो मोटर चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है आजकल अमेरिका की प्रमुख सड़कों का वित्त पोषण करने के लिए पुराने हो चुके सिस्टम का जीर्णोद्धार करने के लिए वाशिंगटन 

In [82]:
print('Avg Bleu Score Obtained for English-Hindi Translation: ', str(Avg_bleu/100))

Avg Bleu Score Obtained for English-Hindi Translation:  0.07343509949563803


**Average Bleu Score Obtained for English - Hindi Translation - 0.07343509949563803**



# Hindi to English

## Data Preparation

In [0]:
# Load the datasets.
dataset=pd.read_csv('train_english.en',names= ['english'],sep='\t',header=None)
dataset1=pd.read_csv('train_hindi.hi',names= ['hindi'],sep='\t',header=None)

test_dataset = pd.read_csv('test_english.en',names= ['english'],sep='\t',header=None)
test_dataset1=pd.read_csv('test_hindi.hi',names= ['hindi'],sep='\t',header=None)


In [0]:
dataset['hindi'] = dataset1['hindi']
test_dataset['hindi'] = test_dataset1['hindi']

In [0]:
# Add sentence beginning and end markers.
dataset['hindi']=dataset['hindi'].apply(lambda x:x+' $')
dataset['english']=dataset['english'].apply(lambda x:'^ '+x+' $')
dataset.head()

test_dataset['hindi']=test_dataset['hindi'].apply(lambda x:x+' $')
test_dataset['english']=test_dataset['english'].apply(lambda x:'^ '+x+' $')


In [78]:
dataset.head()

,english,hindi,e_length,h_length
0,^ give your application an accessibility worko...,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें $,8,9
1,^ accerciser accessibility explorer $,एक्सेर्साइसर पहुंचनीयता अन्वेषक $,5,4
2,^ the default plugin layout for the bottom pan...,निचले पटल के लिए डिफोल्ट प्लग इन खाका $,10,9
3,^ the default plugin layout for the top panel $,ऊपरी पटल के लिए डिफोल्ट प्लग इन खाका $,10,9
4,^ a list of plugins that are disabled by defau...,उन प्लग इनों की सूची जिन्हें डिफोल्ट रूप से नि...,11,14


In [79]:
test_dataset.head()

,english,hindi,e_length,h_length
0,^ a black box in your car $,आपकी कार में ब्लैक बॉक्स $,8,6
1,^ as americas road planners struggle to find t...,जबकि अमेरिका के सड़क योजनाकार ध्वस्त होते हुए ...,38,48
2,^ the devices which track every mile a motoris...,यह डिवाइस जो मोटर चालक द्वारा वाहन चलाए गए प्र...,41,60
3,^ the usually dull arena of highway planning h...,आम तौर पर हाईवे नियोजन जैसा उबाऊ काम भी अचानक ...,17,21
4,^ libertarians have joined environmental group...,आपने द्वारा ड्राइव किए गए मील तथा संभवतः ड्राइ...,41,54


## Data Preprocessing

In [0]:
Y=dataset.english.tolist() + test_dataset.english.tolist()
X=dataset.hindi.tolist() + test_dataset.hindi.tolist()

In [54]:
print(Y)

['^ give your application an accessibility workout $', '^ accerciser accessibility explorer $', '^ the default plugin layout for the bottom panel $', '^ the default plugin layout for the top panel $', '^ a list of plugins that are disabled by default $', '^ highlight duration $', '^ the duration of the highlight box when selecting accessible nodes $', '^ highlight border color $', '^ the color and opacity of the highlight border $', '^ highlight fill color $', '^ the color and opacity of the highlight fill $', '^ api browser $', '^ browse the various methods of the current accessible $', '^ hide private attributes $', '^ method $', '^ property $', '^ value $', '^ ipython console $', '^ interactive console for manipulating currently selected accessible $', '^ event monitor $', '^ monitor events $', '^ clear selection $', '^ everything $', '^ selected application $', '^ selected accessible $', '^ source $', '^ shows events as they occur from selected types and sources $', '^ highlight la

In [0]:
# Compute word counts.
X_words_counts={}
for row in X:
    for words in row.split(' '):
        #print(words)
        X_words_counts[words]=X_words_counts.get(words,0)+1

In [56]:
# Select the most common words.
most_common_X_words = sorted(X_words_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_X_words)

[('$', 2267), ('को', 366), ('के', 336)]


In [0]:
Y_words_counts={}
for row in Y:
    for words in row.split(' '):
        #print(words)
        Y_words_counts[words]=Y_words_counts.get(words,0)+1

In [58]:
most_common_Y_words = sorted(Y_words_counts.items(), key=lambda x: x[1], reverse=True)[:3]
print(most_common_Y_words)

[('^', 2267), ('$', 2267), ('the', 619)]


In [59]:
cnt=0
X_WORDS_TO_INDEX={}
for w in X_words_counts:
    X_WORDS_TO_INDEX[w] =cnt
    cnt+=1
X_WORDS_TO_INDEX['#']=len(X_WORDS_TO_INDEX)
ALL_X_WORDS = X_WORDS_TO_INDEX.keys()
print(X_WORDS_TO_INDEX)

{'अपने': 0, 'अनुप्रयोग': 1, 'को': 2, 'पहुंचनीयता': 3, 'व्यायाम': 4, 'का': 5, 'लाभ': 6, 'दें': 7, '$': 8, 'एक्सेर्साइसर': 9, 'अन्वेषक': 10, 'निचले': 11, 'पटल': 12, 'के': 13, 'लिए': 14, 'डिफोल्ट': 15, 'प्लग': 16, 'इन': 17, 'खाका': 18, 'ऊपरी': 19, 'उन': 20, 'इनों': 21, 'की': 22, 'सूची': 23, 'जिन्हें': 24, 'रूप': 25, 'से': 26, 'निष्क्रिय': 27, 'किया': 28, 'गया': 29, 'है': 30, 'अवधि': 31, 'हाइलाइट': 32, 'रकें': 33, 'पहुंचनीय': 34, 'आसंधि': 35, 'नोड': 36, 'चुनते': 37, 'समय': 38, 'बक्से': 39, 'सीमांत': 40, 'बोर्डर': 41, 'रंग': 42, 'करें': 43, 'किए': 44, 'गए': 45, 'और': 46, 'अपारदर्शिता।': 47, 'भराई': 48, 'पारदर्शिता।': 49, 'एपीआई': 50, 'विचरक': 51, 'इस': 52, 'जिसे': 53, 'प्राप्त': 54, 'हो': 55, 'उसकी': 56, 'विभिन्न': 57, 'विधियों': 58, 'मेथड': 59, 'में': 60, 'विचरण': 61, 'निजी': 62, 'गुणों': 63, 'छिपाएं': 64, 'विधि': 65, 'गुणधर्म': 66, 'मान': 67, 'आईपाइथन': 68, 'कन्सोल': 69, 'चुने': 70, 'एक्सेसेबेल': 71, 'काम': 72, 'लेने': 73, 'अंतर्क्रियात्मक': 74, 'घटना': 75, 'मानिटर': 76, 'घटनाओं': 77, 'चु

In [60]:
cnt=0
Y_WORDS_TO_INDEX={}
for w in Y_words_counts:
    Y_WORDS_TO_INDEX[w] =cnt
    cnt+=1
Y_WORDS_TO_INDEX['#']=len(Y_WORDS_TO_INDEX)
ALL_Y_WORDS = Y_WORDS_TO_INDEX.keys()
print(Y_WORDS_TO_INDEX)

{'^': 0, 'give': 1, 'your': 2, 'application': 3, 'an': 4, 'accessibility': 5, 'workout': 6, '$': 7, 'accerciser': 8, 'explorer': 9, 'the': 10, 'default': 11, 'plugin': 12, 'layout': 13, 'for': 14, 'bottom': 15, 'panel': 16, 'top': 17, 'a': 18, 'list': 19, 'of': 20, 'plugins': 21, 'that': 22, 'are': 23, 'disabled': 24, 'by': 25, 'highlight': 26, 'duration': 27, 'box': 28, 'when': 29, 'selecting': 30, 'accessible': 31, 'nodes': 32, 'border': 33, 'color': 34, 'and': 35, 'opacity': 36, 'fill': 37, 'api': 38, 'browser': 39, 'browse': 40, 'various': 41, 'methods': 42, 'current': 43, 'hide': 44, 'private': 45, 'attributes': 46, 'method': 47, 'property': 48, 'value': 49, 'ipython': 50, 'console': 51, 'interactive': 52, 'manipulating': 53, 'currently': 54, 'selected': 55, 'event': 56, 'monitor': 57, 'events': 58, 'clear': 59, 'selection': 60, 'everything': 61, 'source': 62, 'shows': 63, 'as': 64, 'they': 65, 'occur': 66, 'from': 67, 'types': 68, 'sources': 69, 'last': 70, 'entry': 71, 'start': 

In [61]:
def length_of_sentence(sentence):
    return len(sentence.split(' '))
dataset['e_length']=dataset['english'].apply(length_of_sentence)
dataset['h_length']=dataset['hindi'].apply(length_of_sentence)
Ty=dataset.e_length.max()
Tx=dataset.h_length.max()
print(Tx)
print(Ty)

39
41


In [62]:
def length_of_sentence(sentence):
    return len(sentence.split(' '))
test_dataset['e_length']=test_dataset['english'].apply(length_of_sentence)
test_dataset['h_length']=test_dataset['hindi'].apply(length_of_sentence)
Tyy=test_dataset.e_length.max()
Txx=test_dataset.h_length.max()
print(Txx)
print(Tyy)

60
50


In [0]:
Tx = max(Tx,Txx)
Ty = max(Ty,Tyy)

In [0]:
def series_to_array(series,vocab,length):
    X_train=[]
    for row in series:
        r=row.split(' ')
        R=[]
        for a in r:
            R.append(vocab[a])
        X_train.append(R)
    # length = max(map(len, X_train))
    y=np.array([xi+[None]*(length-len(xi)) for xi in X_train])
    return y

In [65]:
X_train=series_to_array(dataset.hindi,X_WORDS_TO_INDEX,Tx)
X_train=np.where(X_train==None, len(X_WORDS_TO_INDEX)-1, X_train)
print(X_train.shape)

(2167, 60)


In [66]:
Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(X_WORDS_TO_INDEX)), X_train)))
print(Xoh.shape)

(2167, 60, 2126)


In [67]:
Y_train=series_to_array(dataset.english,Y_WORDS_TO_INDEX,Ty)
Y_train=np.where(Y_train==None, len(Y_WORDS_TO_INDEX)-1, Y_train)
print(Y_train)
Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(Y_WORDS_TO_INDEX)), Y_train)))
Yo=Yoh[:,1:,:]
print(Yo.shape)
ze=np.zeros((1,len(Y_WORDS_TO_INDEX)))
ze[0][Y_WORDS_TO_INDEX['^']]=1
Yo=np.insert(arr=Yo,obj=Ty-1,values=ze,axis=1)
print(Yo.shape)

[[0 1 2 ... 1950 1950 1950]
 [0 8 5 ... 1950 1950 1950]
 [0 10 11 ... 1950 1950 1950]
 ...
 [0 1075 7 ... 1950 1950 1950]
 [0 34 7 ... 1950 1950 1950]
 [0 1075 201 ... 1950 1950 1950]]
(2167, 49, 1951)
(2167, 50, 1951)


## Encoder-Decoder Model

In [0]:
n_a=2048

In [69]:
encoder_inputs = Input(shape=(Tx, len(X_WORDS_TO_INDEX)))
encoder = Bidirectional(LSTM(n_a, return_state=True))
encoder_outputs, state_h_f, state_c_f,state_h_b,state_c_b = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
state_h=Concatenate()([state_h_f,state_h_b])
state_c=Concatenate()([state_c_f,state_c_b])
print(state_h.shape)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, len(Y_WORDS_TO_INDEX)))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(2*n_a, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(len(Y_WORDS_TO_INDEX), activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)
print(decoder_outputs.shape)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

(None, 4096)
(None, None, 1951)


In [70]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 60, 2126)     0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) [(None, 4096), (None 68403200    input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 1951)   0                                            
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 4096)         0           bidirectional_2[0][1]            
                                                                 bidirectional_2[0][3]      

In [71]:
opt = Adam(lr=0.001,beta_1=0.9,beta_2=0.999,decay=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit([Xoh,Yoh], Yo, epochs=50, batch_size=64)

Epoch 1/50
2167/2167 [==============================] - 118s 54ms/step - loss: 1.4645 - accuracy: 0.8673
Epoch 2/50
2167/2167 [==============================] - 117s 54ms/step - loss: 0.8792 - accuracy: 0.8932
Epoch 3/50
2167/2167 [==============================] - 117s 54ms/step - loss: 0.7201 - accuracy: 0.9014
Epoch 4/50
2167/2167 [==============================] - 117s 54ms/step - loss: 0.6786 - accuracy: 0.9015
Epoch 5/50
2167/2167 [==============================] - 117s 54ms/step - loss: 0.6525 - accuracy: 0.9025
Epoch 6/50
2167/2167 [==============================] - 117s 54ms/step - loss: 0.6295 - accuracy: 0.9072
Epoch 7/50
2167/2167 [==============================] - 117s 54ms/step - loss: 0.6220 - accuracy: 0.9069
Epoch 8/50
2167/2167 [==============================] - 118s 54ms/step - loss: 0.6118 - accuracy: 0.9079
Epoch 9/50
2167/2167 [==============================] - 118s 54ms/step - loss: 0.5990 - accuracy: 0.9095
Epoch 10/50
2167/2167 [==============================] 

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(2*n_a,))
decoder_state_input_c = Input(shape=(2*n_a,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

## Model Testing

In [0]:
INDEX_TO_WORD_Y={y:x for (x,y) in Y_WORDS_TO_INDEX.items()}


In [74]:
X_test=series_to_array(test_dataset.hindi,X_WORDS_TO_INDEX,Tx)
X_test=np.where(X_test==None, len(X_WORDS_TO_INDEX)-1, X_test)
print(X_test.shape)

from keras.utils import to_categorical
Xoh_test = np.array(list(map(lambda x: to_categorical(x, num_classes=len(X_WORDS_TO_INDEX)), X_test)))
print(Xoh_test.shape)

(100, 60)
(100, 60, 2126)


In [75]:
Y_test=series_to_array(test_dataset.english,Y_WORDS_TO_INDEX,Ty)
Y_test=np.where(Y_test==None, len(Y_WORDS_TO_INDEX)-1, Y_test)
print(Y_test)
Yoh_test = np.array(list(map(lambda x: to_categorical(x, num_classes=len(Y_WORDS_TO_INDEX)), Y_test)))
Yo_test=Yoh_test[:,1:,:]
print(Yo_test.shape)
ze=np.zeros((1,len(Y_WORDS_TO_INDEX)))
ze[0][Y_WORDS_TO_INDEX['^']]=1
Yo_test=np.insert(arr=Yo_test,obj=Ty-1,values=ze,axis=1)
print(Yo_test.shape)

[[0 18 508 ... 1950 1950 1950]
 [0 64 1351 ... 1950 1950 1950]
 [0 10 1363 ... 1950 1950 1950]
 ...
 [0 1743 35 ... 1950 1950 1950]
 [0 1939 186 ... 1950 1950 1950]
 [0 1881 1945 ... 1950 1950 1950]]
(100, 49, 1951)
(100, 50, 1951)


In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(Y_WORDS_TO_INDEX)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, Y_WORDS_TO_INDEX['^']] = 1

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    cnt = 0
    while not stop_condition and cnt < 100:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = INDEX_TO_WORD_Y[sampled_token_index]
        decoded_sentence += (' ' + sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '$' or sampled_char == '#' ):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(Y_WORDS_TO_INDEX)))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]
        cnt+=1

    return decoded_sentence

In [77]:
import nltk
cunt=0
Avg_bleu2 = 0
for index, row in test_dataset.iterrows():
    source=Xoh_test[index,:,:]
    source=source.reshape(1,Tx,len(X_WORDS_TO_INDEX))
    #print(source)
    result=decode_sequence(source)

    hypothesis = result.split(' ')[1:-1]
    reference = row['english'].split(' ')[1:-1]
    print(hypothesis)
    print(reference)
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
    print(BLEUscore)
    Avg_bleu2 += BLEUscore

    # print('##########')
    print("source:", row['hindi'])
    print('real:',row['english'])
    print("output:", result)
    # if cunt==10:
    #     break
    cunt+=1
    print('-------------------------------------------------------------')

print(Avg_bleu2/100)

['the']
['a', 'black', 'box', 'in', 'your', 'car']
0
source: आपकी कार में ब्लैक बॉक्स $
real: ^ a black box in your car $
output:  the $
-------------------------------------------------------------
['the']
['as', 'americas', 'road', 'planners', 'struggle', 'to', 'find', 'the', 'cash', 'to', 'mend', 'a', 'crumbling', 'highway', 'system', 'many', 'are', 'beginning', 'to', 'see', 'a', 'solution', 'in', 'a', 'little', 'black', 'box', 'that', 'fits', 'neatly', 'by', 'the', 'dashboard', 'of', 'your', 'car']
6.305116760146989e-16
source: जबकि अमेरिका के सड़क योजनाकार ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं वहीं बहुत से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है। $
real: ^ as americas road planners struggle to find the cash to mend a crumbling highway system many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car $
output:  the $
------------------------

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


['the']
['the', 'devices', 'which', 'track', 'every', 'mile', 'a', 'motorist', 'drives', 'and', 'transmit', 'that', 'information', 'to', 'bureaucrats', 'are', 'at', 'the', 'center', 'of', 'a', 'controversial', 'attempt', 'in', 'washington', 'and', 'state', 'planning', 'offices', 'to', 'overhaul', 'the', 'outdated', 'system', 'for', 'funding', 'americas', 'major', 'roads']
3.1391327920480296e-17
source: यह डिवाइस जो मोटर चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है आजकल अमेरिका की प्रमुख सड़कों का वित्त पोषण करने के लिए पुराने हो चुके सिस्टम का जीर्णोद्धार करने के लिए वाशिंगटन और राज्य नियोजन कार्यालय के लिए एक विवादास्पद प्रयास का मुद्दा बन चुका है। $
real: ^ the devices which track every mile a motorist drives and transmit that information to bureaucrats are at the center of a controversial attempt in washington and state planning offices to overhaul the outdated system for funding americas major roads $
output:  the $
----------

In [81]:
print('Avg Bleu Score Obtained for Hindi-English Translation: ', str(Avg_bleu2/100))

Avg Bleu Score Obtained for Hindi-English Translation:  0.0002450288639207374


**Average Bleu Score Obtained for Hindi-English Translation - 0.0002450288639207374**